# 패키지 불러오기

In [1]:
import tensorflow as tf
import model as ml
import data
import numpy as np
import os
import sys

from configs import DEFINES # configs.py 파일의 DEFINES 연관 부분만 사용합니다.

# 사전 만들기

In [2]:
DATA_OUT_PATH = './data_out/' # 내려받은 파일 내에 이미 해당 경로가 만들어져있고, 여기로 파일 출력을 진행할 계획.
data_out_path = os.path.join(os.getcwd(), DATA_OUT_PATH) # 현재 작업 폴더 + /data_out
os.makedirs(data_out_path, exist_ok=True) # 만일 셋팅된 폴더가 존재하지 않을 시 해당 폴더를 생성할 것. 
char2idx,  idx2char, vocabulary_length = data.load_vocabulary() 
# data.py 파일의 load_vocabulary를 사용하여 char2idx, idx2char, vocabulary_length 각각에 해당하는 데이터 받아올 것.

# 데이터 만들기

In [3]:
# data.py 파일의 load_data() 함수 사용하여 트레이닝/이벨류에이션 데이너와 레이블 받아옴.
# data.py 파일의 ~~_processing() 함수 사용하여 각 데이터에 적절한 레이블링 진행.

train_input, train_label, eval_input, eval_label = data.load_data()

# 훈련셋 인코딩 만드는 부분이다.
train_input_enc, train_input_enc_length = data.enc_processing(train_input, char2idx)
# 훈련셋 디코딩 입력 부분 만드는 부분이다.
train_output_dec, train_output_dec_length = data.dec_input_processing(train_label, char2idx) #TODO1 실행 안되어 확인 필요(AttributeError: module 'data' has no attribute 'dec_output_processing)
# 훈련셋 디코딩 출력 부분 만드는 부분이다.
train_target_dec = data.dec_target_processing(train_label, char2idx)

# 평가셋 인코딩 만드는 부분이다.
eval_input_enc, eval_input_enc_length = data.enc_processing(eval_input,char2idx)
# 평가셋 인코딩 만드는 부분이다.
eval_output_dec, eval_output_dec_length = data.dec_input_processing(eval_label, char2idx)
# 평가셋 인코딩 만드는 부분이다.
eval_target_dec = data.dec_target_processing(eval_label, char2idx)

C:\Users\goldh\Anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)
100%|█████████████████████████████████████| 3902/3902 [00:09<00:00, 426.26it/s]


# 체크 포인트 경로 만들기

In [4]:
# 현재 경로'./'에 현재 경로 하부에 체크 포인트를 저장한 디렉토리를 설정한다.
check_point_path = os.path.join(os.getcwd(), DEFINES.check_point_path)
# 디렉토리를 만드는 함수이며 두번째 인자 exist_ok가 True이면 디렉토리가 이미 존재해도 OSError가 발생하지 않는다.
# exist_ok가 False이면 해당 디렉토리가 이미 존재할 시 OSError가 발생한다.
os.makedirs(check_point_path, exist_ok=True)

# 에스티 메이터 구성

In [4]:
# 에스티메이터 구성한다.
# 세부 모델 구성은 위에서 임포트한 model.py 내에 따로 정리되어 있습니다. 
# + 모델 내의 각 층애 잡아둔 configuration은 configs.py에 따로 정리된 내용을 받아와서 작업합니다.
# 에스티메이터 자체는 저희가 수업시간에 안 배운거라 공부가 필요합니다...
# 기본 트레이닝 epoch는 20000으로 설정되어 있습니다.

classifier = tf.estimator.Estimator(
        model_fn=ml.model, # 모델 등록한다.
        model_dir=DEFINES.check_point_path, # 체크포인트 위치 등록한다.
        params={ # 모델 쪽으로 파라메터 전달한다.
            'hidden_size': DEFINES.hidden_size, # 가중치 크기 설정한다.
            'layer_size': DEFINES.layer_size, # 멀티 레이어 층 개수를 설정한다.
            'learning_rate': DEFINES.learning_rate, # 학습율 설정한다. 
            'vocabulary_length': vocabulary_length, # 딕셔너리 크기를 설정한다.
            'embedding_size': DEFINES.embedding_size, # 임베딩 크기를 설정한다.
            'embedding': DEFINES.embedding, # 임베딩 사용 유무를 설정한다.
            'multilayer': DEFINES.multilayer, # 멀티 레이어 사용 유무를 설정한다.
        })

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './data_out/check_point', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000146D7164828>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


# 학습 실행

In [5]:
# 학습 실행
classifier.train(input_fn=lambda:data.train_input_fn(
    train_input_enc, train_output_dec, train_target_dec,  DEFINES.batch_size), steps=DEFINES.train_steps)

# config 만져서 20회 학습시켜보려 했는데, 노트북이 불탈거같아서 1100스텝(5회 가량 학습)에서 멈췄습니다.

Instructions for updating:
Colocations handled automatically by placer.
INFO:tensorflow:Calling model_fn.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Done callin

KeyboardInterrupt: 

# 평가 하기

In [6]:
eval_result = classifier.evaluate(input_fn=lambda:data.eval_input_fn(
    eval_input_enc, eval_output_dec, eval_target_dec,  DEFINES.batch_size))
print('\nEVAL set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-07-15T09:17:02Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./data_out/check_point\model.ckpt-733
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-07-15-09:17:27
INFO:tensorflow:Saving dict for global step 733: accuracy = 0.8040902, global_step = 733, loss = 1.4409983
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 733: ./data_out/check_point\model.ckpt-733

EVAL set accuracy: 0.804



일단 5회가량 학습 후 나타내는 정확도는 이벨셋 기준 80퍼 가량. Colab으로 넘어가서 다시 돌려봐야 하지 않을까.. 합니다.

# 테스트 하기

In [9]:
# 테스트용 데이터 만드는 부분이다.
# 인코딩 부분 만든다.
predic_input_enc, predic_input_enc_length = data.enc_processing(["가끔 궁금해"], char2idx)
# 학습 과정이 아니므로 디코딩 입력은 
# 존재하지 않는다.(구조를 맞추기 위해 넣는다.)
predic_output_dec, predic_output_decLength = data.dec_input_processing([""], char2idx)       
# 학습 과정이 아니므로 디코딩 출력 부분도 
# 존재하지 않는다.(구조를 맞추기 위해 넣는다.)
predic_target_dec = data.dec_target_processing([""], char2idx)      

# 예측을 하는 부분이다.
predictions = classifier.predict(
    input_fn=lambda:data.eval_input_fn(predic_input_enc, predic_output_dec, predic_target_dec, DEFINES.batch_size))

# 예측한 값을 인지 할 수 있도록 
# 텍스트로 변경하는 부분이다.
data.pred2string(predictions, idx2char)

100%|████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./data_out/check_point\model.ckpt-733
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
['잘', '<END>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>', '<PADDING>']
잘 


'잘 '

학습이 턱없이 부족해서 그런지, 가끔 궁금해(질문) -> 잘(대답)으로 처리하는 모습입니다. 